In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [5]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [6]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [7]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [9]:
features_list = ['Num_Enq_all_loans_last_9M',
'num_PERSONAL LOAN_enq_last_9M',
'Total_outstanding_Balance_CREDIT CARD_opened_last_9M',
'highest_sanction_amount_credit_cards',
'Age_of_oldest_open_account',
'num_PL_LE_1_Lac_opened_in_last_12M',
'total_emi',
# Reconcilated Features
'months_since_most_recent_delinquency_installment',
 'percentage_utilization_gt_75',
'No of enq in past 90days',
 'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',
                
#ID Features
'dm206s',
'dm216s',
'g406s',
'month_year',
'G15_B24']

In [10]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3038891/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3038891/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [12]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 15), (13738, 15), (20606,), (13738,))

In [13]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 15) (4122, 15) (13738, 15)


In [14]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [15]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,
    'lambda': 6,
    'alpha': 4.5,
    'gamma': 4.8,
    'scale_pos_weight': 2,
    'random_state': 42
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 200,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.58414	eval-auc:0.54970
[10]	train-auc:0.62806	eval-auc:0.60493
[20]	train-auc:0.63407	eval-auc:0.61176
[30]	train-auc:0.63531	eval-auc:0.61295
[40]	train-auc:0.63757	eval-auc:0.61386
[50]	train-auc:0.63889	eval-auc:0.61435
[60]	train-auc:0.64066	eval-auc:0.61557
[70]	train-auc:0.64110	eval-auc:0.61739
[80]	train-auc:0.64123	eval-auc:0.61909
[90]	train-auc:0.64191	eval-auc:0.61849
[100]	train-auc:0.64245	eval-auc:0.61818
[110]	train-auc:0.64365	eval-auc:0.61878
[120]	train-auc:0.64441	eval-auc:0.61811
[123]	train-auc:0.64456	eval-auc:0.61812


In [16]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.64, Gini: 0.29, KS: 0.21
Eval AUC: 0.62, Gini: 0.24, KS: 0.19
Test AUC: 0.63, Gini: 0.26, KS: 0.19
OOT AUC: 0.63, Gini: 0.26, KS: 0.18


In [17]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.147968         0.203152        0.253100        0.184955   1649   
9          0.092233         0.173530        0.184945        0.163998   1648   
8          0.081311         0.155699        0.163995        0.147526   1648   
7          0.064888         0.139733        0.147521        0.132620   1649   
6          0.053398         0.127433        0.132611        0.124323   1648   
5          0.059502         0.122812        0.124319        0.121403   1647   
4          0.048485         0.120186        0.121394        0.119082   1650   
3          0.051376         0.118063        0.119080        0.117037   1635   
2          0.036725         0.115970        0.117033        0.114692   1661   
1          0.018799         0.112440        0.114688        0.109076   1649   

        label_good  label_bad  
Decile                         
10            1405        244  
9             1496        152  
8             1514        134  
7             1542        107  
6             1560         88  
5             1549         98  
4             1570         80  
3             1551         84  
2             1600         61  
1             1618         31

In [18]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.120815         0.203921        0.253746        0.184637   1374   
9          0.087336         0.173725        0.184611        0.163964   1374   
8          0.072780         0.155310        0.163960        0.147052   1374   
7          0.068463         0.139729        0.147049        0.132641   1373   
6          0.061863         0.127667        0.132628        0.124561   1374   
5          0.056041         0.122920        0.124559        0.121533   1374   
4          0.048798         0.120353        0.121532        0.119260   1373   
3          0.042940         0.118146        0.119258        0.117113   1374   
2          0.030590         0.116059        0.117108        0.114800   1373   
1          0.023273         0.112580        0.114798        0.109076   1375   

        label_good  label_bad  
Decile                         
10            1208        166  
9             1254        120  
8             1274        100  
7             1279         94  
6             1289         85  
5             1297         77  
4             1306         67  
3             1315         59  
2             1331         42  
1             1343         32

In [19]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.138760         0.204085        0.250087        0.187139   1081   
9          0.111111         0.175141        0.187099        0.164634   1080   
8          0.096296         0.156196        0.164624        0.147232   1080   
7          0.079630         0.139548        0.147229        0.132706   1080   
6          0.074074         0.127808        0.132693        0.124672   1080   
5          0.075926         0.123070        0.124669        0.121778   1080   
4          0.072222         0.120577        0.121778        0.119506   1080   
3          0.050476         0.118448        0.119505        0.117397   1050   
2          0.039640         0.116363        0.117394        0.115494   1110   
1          0.023127         0.113064        0.115488        0.109076   1081   

        label_good  label_bad  
Decile                         
10             931        150  
9              960        120  
8              976        104  
7              994         86  
6             1000         80  
5              998         82  
4             1002         78  
3              997         53  
2             1066         44  
1             1056         25

In [20]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
14                                              g406s       106.0
4                          Age_of_oldest_open_account        49.0
10                          Num_Enq_non_BANK_last_12M        48.0
0                           Num_Enq_all_loans_last_9M        42.0
8                        percentage_utilization_gt_75        26.0
3                highest_sanction_amount_credit_cards        22.0
2   Total_outstanding_Balance_CREDIT CARD_opened_l...        21.0
5                  num_PL_LE_1_Lac_opened_in_last_12M        18.0
7    months_since_most_recent_delinquency_installment        12.0
1                       num_PERSONAL LOAN_enq_last_9M         9.0
9                            No of enq in past 90days         7.0
12                                             dm206s         5.0
13                                             dm216s         2.0
6                                           total_emi         1.0
11        

In [21]:
feature_importance_df

Feature  Importance
14                                              g406s       106.0
4                          Age_of_oldest_open_account        49.0
10                          Num_Enq_non_BANK_last_12M        48.0
0                           Num_Enq_all_loans_last_9M        42.0
8                        percentage_utilization_gt_75        26.0
3                highest_sanction_amount_credit_cards        22.0
2   Total_outstanding_Balance_CREDIT CARD_opened_l...        21.0
5                  num_PL_LE_1_Lac_opened_in_last_12M        18.0
7    months_since_most_recent_delinquency_installment        12.0
1                       num_PERSONAL LOAN_enq_last_9M         9.0
9                            No of enq in past 90days         7.0
12                                             dm206s         5.0
13                                             dm216s         2.0
6                                           total_emi         1.0
11                     Num_of_installment_loan_opened         1.0

In [22]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BI_Train_Pred_9024.csv',index=False)
Eval.to_csv('LeoPayu_BI_Eval_Pred_9024.csv',index=False)
Test.to_csv('LeoPayu_BI_Test_Pred_9024.csv',index=False)
Oot.to_csv('LeoPayu_BI_Oot_Pred_9024.csv',index=False)


In [23]:
#Save the model
xgb_model.save_model('LeoPayu_BI_15_in_24mob.json')
print('Model Saved')

Model Saved
